In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
data=pd.read_csv("mnist_test.csv")
data.head()

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
data=np.array(data)
m,n=data.shape
np.random.shuffle(data)

data_dev=data[0:1000].T #each column is a training example now.
X_dev=data_dev[1:n]
y_dev=data_dev[0]

data_train=data[1000:m].T
X_train=data_train[1:n]
y_train=data_train[0]

X_train = X_train / 255.0
X_dev = X_dev / 255.0

In [28]:
def init_params():
    # Layer 1: input (784) -> hidden (512)
    W1 = np.random.randn(512, 784) * np.sqrt(1/784)
    b1 = np.zeros((512, 1))
    
    # Layer 2: hidden (512) -> hidden (128)
    W2 = np.random.randn(128, 512) * np.sqrt(1/512)
    b2 = np.zeros((128, 1))
    
    # Layer 3: hidden (128) -> hidden (64)  (optional extra hidden layer)
    W3 = np.random.randn(64, 128) * np.sqrt(1/128)
    b3 = np.zeros((64, 1))
    
    # Layer 4: hidden (64) -> output (10)
    W4 = np.random.randn(10, 64) * np.sqrt(1/64)
    b4 = np.zeros((10, 1))
    
    return W1, b1, W2, b2, W3, b3, W4, b4



def ReLU(Z):
  return np.maximum(Z,0)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return expZ / np.sum(expZ, axis=0, keepdims=True)

def forward_prop(W1, b1, W2, b2, W3, b3, W4, b4, X):
    # Layer 1
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    
    # Layer 2
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    
    # Layer 3
    Z3 = W3.dot(A2) + b3
    A3 = ReLU(Z3)
    
    # Layer 4 (Output)
    Z4 = W4.dot(A3) + b4
    A4 = softmax(Z4)
    
    return Z1, A1, Z2, A2, Z3, A3, Z4, A4


def one_hot(y):
  one_hot_y=np.zeros((y.size,y.max()+1))
  one_hot_y[np.arange(y.size),y]=1
  one_hot_y=one_hot_y.T
  return one_hot_y

def ReLU_deriv(Z):
  return Z>0

def back_prop(Z1, A1, Z2, A2, W1, W2, W3, W4, X, y):
    one_hot_y = one_hot(y)
    m = y.size
    dZ2 = A2 - one_hot_y
    dW2 = (1 / m) * dZ2.dot(A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = (1 / m) * dZ1.dot(X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def back_prop(Z1, A1, Z2, A2, Z3, A3, Z4, A4, 
              W1, W2, W3, W4, X, y):
    one_hot_y = one_hot(y)
    m = y.size

    # Layer 4 (Softmax + Cross-Entropy)
    dZ4 = A4 - one_hot_y
    dW4 = (1 / m) * dZ4.dot(A3.T)
    db4 = (1 / m) * np.sum(dZ4, axis=1, keepdims=True)

    # Layer 3 (ReLU)
    dZ3 = W4.T.dot(dZ4) * ReLU_deriv(Z3)
    dW3 = (1 / m) * dZ3.dot(A2.T)
    db3 = (1 / m) * np.sum(dZ3, axis=1, keepdims=True)

    # Layer 2 (ReLU)
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = (1 / m) * dZ2.dot(A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)

    # Layer 1 (ReLU)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = (1 / m) * dZ1.dot(X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2, dW3, db3, dW4, db4

def update_params(W1, b1, W2, b2, W3, b3, W4, b4,
                  dW1, db1, dW2, db2, dW3, db3, dW4, db4,
                  alpha):
    # Layer 1
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    
    # Layer 2
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    
    # Layer 3
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3
    
    # Layer 4
    W4 = W4 - alpha * dW4
    b4 = b4 - alpha * db4
    
    return W1, b1, W2, b2, W3, b3, W4, b4



In [29]:
def get_predictions(A4):
    return np.argmax(A4, 0)

def get_accuracy(predictions, Y):
    correct = np.sum(predictions == Y)
    accuracy = correct / Y.size * 100  # float, always between 0 and 100
    print(f"Predictions: {predictions}, Labels: {Y}")
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy


def gradient_descent(X, Y, alpha, iterations):
    # Initialize parameters
    W1, b1, W2, b2, W3, b3, W4, b4 = init_params()

    for i in range(iterations):
        # Forward pass
        Z1, A1, Z2, A2, Z3, A3, Z4, A4 = forward_prop(W1, b1, W2, b2, W3, b3, W4, b4, X)

        # Backward pass
        dW1, db1, dW2, db2, dW3, db3, dW4, db4 = back_prop(
            Z1, A1, Z2, A2, Z3, A3, Z4, A4, W1, W2, W3, W4, X, Y
        )

        # Update parameters
        W1, b1, W2, b2, W3, b3, W4, b4 = update_params(
            W1, b1, W2, b2, W3, b3, W4, b4,
            dW1, db1, dW2, db2, dW3, db3, dW4, db4, alpha
        )

        # Monitor progress
        if i % 500 == 0:
            print("Iteration:", i)
            predictions = get_predictions(A4)   # <-- output layer predictions
            print("Accuracy:", get_accuracy(predictions, Y))

    return W1, b1, W2, b2, W3, b3, W4, b4


In [30]:
W1, b1, W2, b2, W3, b3, W4, b4 = gradient_descent(X_train, y_train, 0.1, 2000)

Iteration: 0
Predictions: [7 7 3 ... 7 7 1], Labels: [8 1 6 ... 0 1 8]
Accuracy: 9.93%
Accuracy: 9.934437159684409
Iteration: 500
Predictions: [8 1 6 ... 0 1 8], Labels: [8 1 6 ... 0 1 8]
Accuracy: 96.96%
Accuracy: 96.9552172463607
Iteration: 1000
Predictions: [8 1 6 ... 0 1 8], Labels: [8 1 6 ... 0 1 8]
Accuracy: 99.38%
Accuracy: 99.37770863429269
Iteration: 1500
Predictions: [8 1 6 ... 0 1 8], Labels: [8 1 6 ... 0 1 8]
Accuracy: 99.97%
Accuracy: 99.9666629625514


In [31]:
def make_predictions(X, W1, b1, W2, b2, W3, b3, W4, b4):
    # Forward pass (get all outputs, but only need A4)
    _, _, _, _, _, _, _, A4 = forward_prop(W1, b1, W2, b2, W3, b3, W4, b4, X)
    
    # Get predicted class labels
    predictions = get_predictions(A4)
    return predictions

def test_prediction(index, W1, b1, W2, b2, W3, b3, W4, b4):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2,W3, b3, W4, b4)
    label = y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

In [32]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2, W3, b3, W4, b4)
get_accuracy(dev_predictions, y_dev)

Predictions: [2 0 4 4 0 7 7 2 7 1 7 3 3 5 3 1 9 1 5 3 0 1 5 9 1 3 3 6 2 7 4 1 3 4 8 5 9
 4 8 8 7 5 6 4 9 6 3 7 8 9 5 7 2 3 8 9 0 1 1 3 1 7 1 8 2 2 1 9 2 5 4 1 0 2
 1 0 8 9 6 4 4 3 4 0 1 1 1 6 9 4 9 6 9 1 5 9 7 8 0 0 4 2 1 6 5 4 3 6 8 5 8
 0 3 8 2 0 0 9 3 4 2 8 9 1 4 0 1 8 4 7 2 0 5 1 7 3 3 0 8 0 7 5 9 3 3 8 3 1
 5 8 1 0 7 8 5 1 7 6 1 3 4 2 1 5 5 5 4 1 1 5 5 0 8 2 1 5 8 9 0 0 3 2 4 7 6
 8 2 5 0 4 7 9 5 9 6 2 4 1 3 9 9 5 3 3 1 0 7 4 0 5 1 6 1 3 9 1 6 9 5 0 4 2
 2 8 2 9 5 8 7 7 8 2 5 3 7 7 2 9 4 8 9 8 1 2 0 1 2 7 9 9 0 2 6 1 9 7 3 1 0
 4 5 2 2 7 3 3 9 0 4 4 4 5 9 5 1 5 9 6 1 6 6 1 0 0 1 1 5 1 7 2 2 1 2 1 1 7
 3 3 8 3 5 9 3 4 8 1 6 1 9 1 5 5 2 5 7 0 7 2 1 8 8 1 0 4 4 9 1 2 4 7 7 7 6
 4 2 3 4 2 9 0 1 1 5 5 9 3 9 0 1 3 3 6 8 5 5 7 7 9 1 2 0 9 0 8 2 2 7 8 3 6
 5 8 6 2 5 1 1 3 3 3 3 6 4 1 6 3 6 4 1 7 1 0 1 8 0 4 0 0 2 3 6 1 1 4 4 8 9
 2 2 6 2 5 8 8 0 3 9 4 2 9 0 8 1 5 2 0 2 0 6 9 4 1 7 0 7 7 4 4 8 1 1 0 8 1
 8 6 4 5 4 8 1 2 2 9 2 0 5 8 0 6 5 5 0 2 9 3 2 4 3 2 7 9 1 5 7 3 3 5 8 7 3
 4 1 9 3 8 3

95.8